In [3]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd


data_coor = pd.read_csv('Geospatial_Coordinates.csv')
data_coor.columns = ['PostalCode', 'Latitude', 'Longitude']

with open('ListaCANADA.html') as html_file:
#with open('http://coreyms.com') as html_file:
    soup=BeautifulSoup(html_file,'lxml')

data = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'])

j=1  
for city in soup.findAll('table',class_="wikitable sortable"):

    PostalCodeOld=''
    BoroughOld=''
    NeighborhoodOld=''

    PostalCodeNew=''
    BoroughNew=''
    NeighborhoodNew=''
    
 #   Insertar=0
    
    for row in city.find_all('tr'):
        i=1
        for row1 in row.find_all('td'):
            try:
                #print(row1.text)
                if i==1:
                    PostalCodeNew=row1.text
                else:    
                    if i==2:
                        BoroughNew=row1.text
                    else:
                        if i==3:
                            NeighborhoodNew=row1.text.replace("\n","")
                            if NeighborhoodNew=="Not assigned":
                                NeighborhoodNew=BoroughNew

            except Exception as e:    
                row1=None
           
            i=i+1
           
        if PostalCodeNew!='' and BoroughNew!="Not assigned":
            if PostalCodeNew==PostalCodeOld:
                NeighborhoodNew=NeighborhoodOld+";"+NeighborhoodNew
                data.at[[j - 1],"Neighborhood"]= NeighborhoodNew
            else:
                Latitude=data_coor.loc[data_coor['PostalCode'] == PostalCodeNew,['Latitude']].to_string(header = None,index=None)
                Longitude=data_coor.loc[data_coor['PostalCode'] == PostalCodeNew,['Longitude']].to_string(header = None,index=None)
                data.loc[j] = [PostalCodeNew, BoroughNew, NeighborhoodNew,Latitude,Longitude]
                j=j+1
           
        PostalCodeOld=PostalCodeNew
        BoroughOld=BoroughNew
        NeighborhoodOld=NeighborhoodNew

data.shape
data.head()



,PostalCode,Borough,Neighborhood,Latitude,Longitude
1,M3A,North York,Parkwoods,43.753259,-79.329656
2,M4A,North York,Victoria Village,43.725882,-79.315572
3,M5A,Downtown Toronto,Harbourfront;Regent Park,43.65426,-79.360636
4,M6A,North York,Lawrence Heights;Lawrence Manor,43.718518,-79.464763
5,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


'-79.188711'